### Praktikum 3
Install hnswlib terlebih dahulu.

In [ ]:
!pip install hnswlib

Percobaan berikut akan membandingkan exact NN dengan HNSW pada 1000 data 2D.

In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[847 227 738 467 325]]
Distances: [[0.01786956 0.01925608 0.03536841 0.03579538 0.03946245]]
Waktu: 0.04342389106750488 detik

=== HNSW ===
Indices: [[847 227 738 467 325]]
Distances: [[0.00031932 0.0003708  0.00125092 0.00128131 0.00155728]]
Waktu: 0.00020623207092285156 detik


Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [ ]:
import numpy as np
import hnswlib
import time
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# --- Konfigurasi Global Eksperimen ---
K_NEIGHBORS = 5 # Jumlah tetangga terdekat yang dicari

# Daftar semua skenario eksperimen yang akan dijalankan
EXPERIMENT_SCENARIOS = [
    # Skenario 2D
    {'dimensions': 2, 'n_points': 1000},
    {'dimensions': 2, 'n_points': 1_000_000},
    # Skenario 5D
    {'dimensions': 5, 'n_points': 1000},
    {'dimensions': 5, 'n_points': 1_000_000},
]

# Parameter spesifik untuk HNSW
HNSW_M = 16
HNSW_EF_CONSTRUCTION = 200
HNSW_EF_SEARCH = 50

def run_sklearn_brute_experiment(X, query, metric):
    """Menjalankan eksperimen untuk Scikit-learn NearestNeighbors (Brute Force)."""
    start_build = time.time()
    nn = NearestNeighbors(n_neighbors=K_NEIGHBORS, algorithm='brute', metric=metric)
    nn.fit(X)
    build_time = time.time() - start_build

    start_search = time.time()
    nn.kneighbors(query)
    search_time = time.time() - start_search
    return build_time, search_time

def run_hnsw_experiment(X, query, space, dimensions):
    """Menjalankan eksperimen untuk HNSWlib."""
    start_build = time.time()
    p = hnswlib.Index(space=space, dim=dimensions)
    p.init_index(max_elements=len(X), ef_construction=HNSW_EF_CONSTRUCTION, M=HNSW_M)
    p.add_items(X)
    build_time = time.time() - start_build

    p.set_ef(HNSW_EF_SEARCH)
    start_search = time.time()
    p.knn_query(query, k=K_NEIGHBORS)
    search_time = time.time() - start_search
    return build_time, search_time

# --- Loop Utama Eksperimen ---
all_results = []
np.random.seed(42)

for scenario in EXPERIMENT_SCENARIOS:
    dims = scenario['dimensions']
    n_pts = scenario['n_points']

    print(f"\n{'='*20} SCENARIO: {n_pts} points, {dims} dimensions {'='*20}")

    # 1. Buat Dataset
    print(f"  Generating dataset...")
    X_raw = np.random.rand(n_pts, dims).astype('float32')
    query_raw = np.random.rand(1, dims).astype('float32')

    # --- Menjalankan untuk setiap metrik ---
    for metric_name in ['euclidean', 'angular']:
        print(f"\n  --- Metric: {metric_name} ---")
        X = X_raw.copy()
        query = query_raw.copy()

        # Penyesuaian metrik antar library
        if metric_name == 'angular':
            # Untuk angular/cosine, kita normalkan datanya
            # HNSW menggunakan 'ip' (inner product), scikit-learn menggunakan 'cosine'
            norm_X = np.linalg.norm(X, axis=1, keepdims=True)
            X = X / norm_X
            norm_query = np.linalg.norm(query, axis=1, keepdims=True)
            query = query / norm_query

            hnsw_space = 'ip'
            sklearn_metric = 'cosine'
        else:
            hnsw_space = 'l2'
            sklearn_metric = 'euclidean'

        # Menjalankan algoritma
        # Scikit-learn Brute Force
        try:
            b_t, s_t = run_sklearn_brute_experiment(X, query, sklearn_metric)
            all_results.append(['Scikit-learn(Brute)', metric_name, dims, n_pts, b_t, s_t * 1000])
            print(f"    Scikit-learn   | Build: {b_t:.4f}s | Search: {s_t*1000:.4f}ms")
        except Exception as e:
            print(f"    Scikit-learn failed: {e}")

        # HNSWlib
        try:
            b_t, s_t = run_hnsw_experiment(X, query, hnsw_space, dims)
            all_results.append(['HNSWlib', metric_name, dims, n_pts, b_t, s_t * 1000])
            print(f"    HNSWlib        | Build: {b_t:.4f}s | Search: {s_t*1000:.4f}ms")
        except Exception as e:
            print(f"    HNSWlib failed: {e}")


# --- Tampilkan Hasil Akhir dalam Bentuk Tabel ---
df_results = pd.DataFrame(all_results, columns=[
    'Algorithm', 'Metric', 'Dimensions', 'Data Points', 'Build Time (s)', 'Search Time (ms)'
])

print("\n\n" + "="*50)
print("              HASIL AKHIR EKSPERIMEN")
print("="*50)
print(df_results.to_string())



==================== SCENARIO: 1000 points, 2 dimensions ====================
  Generating dataset...

  --- Metric: euclidean ---
    Scikit-learn   | Build: 0.0017s | Search: 2.4557ms
    HNSWlib        | Build: 0.1622s | Search: 0.0672ms

  --- Metric: angular ---
    Scikit-learn   | Build: 0.0007s | Search: 9.7730ms
    HNSWlib        | Build: 0.1091s | Search: 0.0560ms

==================== SCENARIO: 1000000 points, 2 dimensions ====================
  Generating dataset...

  --- Metric: euclidean ---
    Scikit-learn   | Build: 0.0018s | Search: 50.2334ms
    HNSWlib        | Build: 87.4527s | Search: 0.0939ms

  --- Metric: angular ---
    Scikit-learn   | Build: 0.0020s | Search: 45.5358ms
    HNSWlib        | Build: 115.6929s | Search: 0.1228ms

==================== SCENARIO: 1000 points, 5 dimensions ====================
  Generating dataset...

  --- Metric: euclidean ---
    Scikit-learn   | Build: 0.0007s | Search: 0.7837ms
    HNSWlib        | Build: 0.0627s | Search: 0